In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
from sklearn.metrics import mean_squared_error

In [9]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
X = housing["data"]
Y = housing["target"]

In [15]:
print(len(X))
print(max(Y))

20640
5.00001


In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
svr = SVR(kernel="rbf")
svr.fit(X_train_scaled, Y_train)
Y_train_pred = svr.predict(X_train_scaled)
np.sqrt(mean_squared_error(Y_train, Y_train_pred))

0.5775936897867326

In [54]:
svr

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [60]:
params_dict = {"gamma": uniform(0.05, 0.1), "C": uniform(0, 5)}
random_search = RandomizedSearchCV(svr, params_dict, n_iter = 24, verbose = 2, cv=3, n_jobs = -1)
random_search.fit(X_train_scaled, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='scale', kernel='rbf',
                                 max_iter=-1, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='deprecated', n_iter=24, n_jobs=-1,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc5ed47d0d0>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc5c31fec50>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [61]:
random_search.best_estimator_

SVR(C=4.568193145274709, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.1475276081267851, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [62]:
Y_train_pred = random_search.best_estimator_.predict(X_train_scaled)
np.sqrt(mean_squared_error(Y_train, Y_train_pred))

0.5440661939974761

In [63]:
Y_test_pred = random_search.best_estimator_.predict(X_test_scaled)
np.sqrt(mean_squared_error(Y_test, Y_test_pred))

0.583002390829906

In [64]:
Y_test_pred = svr.predict(X_test_scaled)
np.sqrt(mean_squared_error(Y_test, Y_test_pred))

0.6045460338971437

### It turns out that default settings were pretty good